In [1]:
import os
import fnmatch as fn
import numpy as np
import pandas as pd
#!pip install dbfread
import dbfread

#!pip install pandas_profiling
import pandas_profiling as pp
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from matplotlib import font_manager, rc
%matplotlib inline
#%matplotlib notebook

import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
from math import sin,cos,sqrt,atan2,radians 
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import shapiro
import tbats
#import fbprophet

import requests
import logging
import time
import googlemaps
import reverse_geocoder
import pprint
import folium
import zipfile
import json
#%matplotlib.inline

matplotlib.rcParams['axes.unicode_minus'] = False # 마이너스 기호도 표시

# 한글 깨짐 방지 목적 #
font_name = font_manager.FontProperties(fname = 'c:/Windows/Fonts/malgun.ttf').get_name()
rc('font',family = font_name)

## 맥
# rc('font', family='/Library/Fonts/AppleGothic.ttf')


# 만약 plotly가 설치 되지 않았을 경우 하단의 코드를 실행할 것 #
#!pip install plotly
import plotly
#!pip install cufflinks
#import plotly.express as px
# Cufflinks wrapper on plotly
import cufflinks as cf
import plotly.graph_objs as go
import plotly.plotly as py
import plotly.figure_factory as ff
from plotly.offline import iplot
from plotly.offline import init_notebook_mode, plot,iplot

init_notebook_mode(connected = True)
cf.go_offline()

import warnings
warnings.filterwarnings(action='ignore') # 경고 메시지를 숨길 때

ModuleNotFoundError: No module named 'tbats'

# Existed Merging form

In [2]:
df = pd.read_csv('가공데이터/merge_유통_유동_환경_소상공인_edit1.csv')
df = df.iloc[:,1:]
df.head()

,date,year,quarter,month,weekday,day,district,p10_most_freq_day,p10_mean_day,p10_25per,...,AMT_IND_60,AMT_IND_70,AMT_IND_80,tp_method1,cri,gs_num,min_gs,max_gs,median_gs,gsindex1
0,2018-04-01,2018,2,4,Sunday,1,가회동,나쁨,66.773843,57.656944,...,0.007142,0.015259,0.000732,17191.22,2018-03-31,1,0.0,0.0,0.0,1.211474
1,2018-04-02,2018,2,4,Monday,2,가회동,나쁨,64.771759,56.862500,...,0.001952,0.001996,0.009537,32378.20,2018-03-31,1,0.0,0.0,0.0,0.997001
2,2018-04-03,2018,2,4,Tuesday,3,가회동,나쁨,63.759838,48.220833,...,0.005172,0.007284,0.007231,35185.26,2018-03-31,1,0.0,0.0,0.0,0.451986
3,2018-04-04,2018,2,4,Wednesday,4,가회동,좋음,18.720500,10.300589,...,0.013560,0.091377,0.003216,34655.78,2018-03-31,1,0.0,0.0,0.0,0.085016
4,2018-04-05,2018,2,4,Thursday,5,가회동,좋음,12.597801,10.083333,...,0.018180,0.030341,0.011542,33459.81,2018-03-31,1,0.0,0.0,0.0,1.012667


In [3]:
df.columns

Index(['date', 'year', 'quarter', 'month', 'weekday', 'day', 'district',
       'p10_most_freq_day', 'p10_mean_day', 'p10_25per', 'p10_median_day',
       'p10_75per', 'p25_most_freq_day', 'p25_mean_day', 'p25_25per',
       'p25_median_day', 'p25_75per', 'humi_mean_day', 'humi_median_day',
       'humi_25per', 'humi_75per', 'temp_mean_day', 'temp_median_day',
       'temp_25per', 'temp_75per', 'noise_mean_day', 'noise_median_day',
       'dis_index_most_freq2', 'MAN_0004', 'MAN_0509', 'MAN_1014', 'MAN_1519',
       'MAN_2024', 'MAN_2529', 'MAN_3034', 'MAN_3539', 'MAN_4044', 'MAN_4549',
       'MAN_5054', 'MAN_5559', 'MAN_6064', 'MAN_6569', 'MAN_70U', 'WMAN_0004',
       'WMAN_0509', 'WMAN_1014', 'WMAN_1519', 'WMAN_2024', 'WMAN_2529',
       'WMAN_3034', 'WMAN_3539', 'WMAN_4044', 'WMAN_4549', 'WMAN_5054',
       'WMAN_5559', 'WMAN_6064', 'WMAN_6569', 'WMAN_70U', 'SGNG_NM', 'values',
       'LENGTH', 'AREA', '10_P', '20_P', '30_P', '40_P', '50_P', '60_P',
       '70_P', '80_P', 'AMT_IND

# New way to merge

In [4]:
gs_retail = pd.read_csv('가공데이터/GS편의점_종로노원구.csv')
gs_retail = gs_retail.iloc[:,1:]

gs_distance = pd.read_csv('가공데이터/gs_distance.csv')
gs_distance = gs_distance.iloc[:,1:]

ad = dbfread.DBF('Rawdata/유동인구데이터/행정동경계파일/종로_노원_행정동.dbf')
ad = pd.DataFrame(iter(ad))

ad['X_COORD'] = ad['X_COORD'].apply(float)
ad['Y_COORD'] = ad['Y_COORD'].apply(float)

g_info = [Point(i) for i in zip(ad['X_COORD'],ad['Y_COORD'])]

geo_ad = gpd.GeoDataFrame(ad,
                          crs = {'init' : 'epsg:4326'},
                          geometry = g_info
                         )
geo_ad.head()

,STD_YYYY,HDONG_CD,HDONG_NM,SIDO_CD,SIDO_NM,SGNG_CD,SGNG_NM,AREA,LENGTH,X_COORD,Y_COORD,geometry
0,2019,1111051500,청운효자동,11,서울특별시,11110,종로구,2573175.62,7975.05,126.972694,37.581306,POINT (126.972694 37.581306)
1,2019,1111053000,사직동,11,서울특별시,11110,종로구,1128161.02,5498.40,126.970919,37.573408,POINT (126.9709195 37.573408)
2,2019,1111054000,삼청동,11,서울특별시,11110,종로구,1478468.25,7377.76,126.984028,37.582083,POINT (126.984028 37.582083)
3,2019,1111055000,부암동,11,서울특별시,11110,종로구,2276588.99,8912.71,126.966500,37.589917,POINT (126.9665 37.589917)
4,2019,1111056000,평창동,11,서울특별시,11110,종로구,8951336.18,13312.43,126.968972,37.602583,POINT (126.968972 37.602583)


## Load raw data(Transient population data) and data prerpocessing

In [5]:
tp = pd.read_csv('가공데이터/유동_edit1.csv')
tp = tp.iloc[:,1:]

In [6]:
tp = tp.loc[:,['STD_YMD','HDONG_NM','MAN_0004', 'MAN_0509',
       'MAN_1014', 'MAN_1519', 'MAN_2024', 'MAN_2529', 'MAN_3034', 'MAN_3539',
       'MAN_4044', 'MAN_4549', 'MAN_5054', 'MAN_5559', 'MAN_6064', 'MAN_6569',
       'MAN_70U', 'WMAN_0004', 'WMAN_0509', 'WMAN_1014', 'WMAN_1519',
       'WMAN_2024', 'WMAN_2529', 'WMAN_3034', 'WMAN_3539', 'WMAN_4044',
       'WMAN_4549', 'WMAN_5054', 'WMAN_5559', 'WMAN_6064', 'WMAN_6569',
       'WMAN_70U']]

In [7]:
tp.columns

Index(['STD_YMD', 'HDONG_NM', 'MAN_0004', 'MAN_0509', 'MAN_1014', 'MAN_1519',
       'MAN_2024', 'MAN_2529', 'MAN_3034', 'MAN_3539', 'MAN_4044', 'MAN_4549',
       'MAN_5054', 'MAN_5559', 'MAN_6064', 'MAN_6569', 'MAN_70U', 'WMAN_0004',
       'WMAN_0509', 'WMAN_1014', 'WMAN_1519', 'WMAN_2024', 'WMAN_2529',
       'WMAN_3034', 'WMAN_3539', 'WMAN_4044', 'WMAN_4549', 'WMAN_5054',
       'WMAN_5559', 'WMAN_6064', 'WMAN_6569', 'WMAN_70U'],
      dtype='object')

In [8]:
we = []
we2 = []
for j in range(2,len(tp.columns),1):
    sh1 = tp.columns.str.split('_')[j][0]
    sh2 = tp.columns.str.split('_')[j][1]
    we.append(sh1)
    we2.append(sh2)

In [9]:
we_new = we * (len(np.unique(tp['STD_YMD'])) * len(np.unique(tp['HDONG_NM'])))
we2_new = we2 * (len(np.unique(tp['STD_YMD'])) * len(np.unique(tp['HDONG_NM'])))
trans_pop = tp.iloc[:,2:].values.ravel()
date = [a for i in tp['STD_YMD'] for a in [i] * len(we)]
dong = [uy for j in tp['HDONG_NM'] for uy in [j] * len(we)]

In [10]:
new_tp = pd.DataFrame({'STD_YMD' : date, 'ADMD_NM' : dong, 'SEX_CD' : we_new, 'AGE_CD' : we2_new, 'trans_pop':trans_pop})

In [11]:
new_tp.head()

,STD_YMD,ADMD_NM,SEX_CD,AGE_CD,trans_pop
0,20180401,청운효자동,MAN,0004,0.05
1,20180401,청운효자동,MAN,0509,78.93
2,20180401,청운효자동,MAN,1014,403.26
3,20180401,청운효자동,MAN,1519,642.66
4,20180401,청운효자동,MAN,2024,984.24


In [12]:
new_tp['SEX_CD'].replace({'MAN' : 'M','WMAN':'F'},inplace=True)

In [13]:
new_tp['AGE_CD'].replace({'0004':'20','0509' :'20','1014':'20','1519' :'20',
                          '2024':'20','2529':'25',
                          '3034':'30','3539':'35',
                          '4044':'40','4549':'45',
                          '5054':'50','5559':'55',
                          '6064':'60','6569':'65','70U' : '65'},inplace=True)

In [14]:
new_tp2 = new_tp.groupby(['STD_YMD','ADMD_NM','SEX_CD','AGE_CD'])['trans_pop'].sum().reset_index(level=[0,1,2,3])
new_tp2['AGE_CD'] = new_tp2['AGE_CD'].astype('int64')
new_tp2.head()

,STD_YMD,ADMD_NM,SEX_CD,AGE_CD,trans_pop
0,20180401,가회동,F,20,1160.64
1,20180401,가회동,F,25,832.24
2,20180401,가회동,F,30,729.24
3,20180401,가회동,F,35,812.86
4,20180401,가회동,F,40,741.41


In [15]:
new_tp2.to_csv('가공데이터/유동_edit2.csv',encoding='utf-8')

## Load raw data(Circulation data) and data prerpocessing

In [16]:
cir = pd.read_csv('가공데이터/유통_edit1.csv')
cir.columns

Index(['time', 'ADMD_NM', 'values', 'year', 'month', 'day', 'weekday',
       'SGNG_NM', 'SGNG_CD', 'HDONG_CD', 'LENGTH', 'AREA', '10_P', '20_P',
       '30_P', '40_P', '50_P', '60_P', '70_P', '80_P', 'AMT_IND_10',
       'AMT_IND_20', 'AMT_IND_30', 'AMT_IND_40', 'AMT_IND_50', 'AMT_IND_60',
       'AMT_IND_70', 'AMT_IND_80'],
      dtype='object')

In [17]:
cir2 = cir.iloc[:,:-8]
cir2 = cir2.loc[:,['time', 'ADMD_NM', 'values',
                   '10_P', '20_P','30_P', '40_P', '50_P', '60_P', '70_P', '80_P']]
cir2.head()

,time,ADMD_NM,values,10_P,20_P,30_P,40_P,50_P,60_P,70_P,80_P
0,2018-04-01,가회동,0.741018,0.238186,0.210444,0.412769,0.016912,0.090471,0.009637,0.020592,0.000988
1,2018-04-02,가회동,0.855520,0.248125,0.214794,0.409670,0.025352,0.086298,0.002281,0.002333,0.011148
2,2018-04-03,가회동,0.981773,0.261384,0.204568,0.347117,0.068550,0.098328,0.005268,0.007419,0.007365
3,2018-04-04,가회동,1.059795,0.230096,0.211954,0.316462,0.043199,0.096238,0.012795,0.086221,0.003035
4,2018-04-05,가회동,0.855990,0.261243,0.136507,0.275644,0.032804,0.223634,0.021238,0.035445,0.013484


In [18]:
we = [cir2.columns.str.split('_')[j][0] + '_cate' for j in range(3,len(cir2.columns),1)]

In [19]:
category = [cir2.columns.str.split('_')[j][0] + '_cate' for j in range(3,len(cir2.columns),1)] * len(np.unique(cir2['ADMD_NM'])) * len(np.unique(cir2['time']))
date = [a for i in cir2['time'] for a in [i] * len(we)]
dong = [u for j in cir2['ADMD_NM'] for u in [j] * len(we)]
dong_index = [z for j in cir2['values'] for z in [j]*len(we)]
values = cir2.iloc[:,3:].values.ravel()

In [20]:
cir_final = pd.DataFrame({'STD_YMD':date,
                          'ADMD_NM': dong, 
                          'values':dong_index,
                           'category' : category,
                          'category_values' : values})

In [21]:
cir_final.head()

,STD_YMD,ADMD_NM,values,category,category_values
0,2018-04-01,가회동,0.741018,10_cate,0.238186
1,2018-04-01,가회동,0.741018,20_cate,0.210444
2,2018-04-01,가회동,0.741018,30_cate,0.412769
3,2018-04-01,가회동,0.741018,40_cate,0.016912
4,2018-04-01,가회동,0.741018,50_cate,0.090471


In [22]:
cir_final['values_ratio'] = cir_final['values'] * cir_final['category_values']
cir_final.head()

,STD_YMD,ADMD_NM,values,category,category_values,values_ratio
0,2018-04-01,가회동,0.741018,10_cate,0.238186,0.176500
1,2018-04-01,가회동,0.741018,20_cate,0.210444,0.155943
2,2018-04-01,가회동,0.741018,30_cate,0.412769,0.305870
3,2018-04-01,가회동,0.741018,40_cate,0.016912,0.012532
4,2018-04-01,가회동,0.741018,50_cate,0.090471,0.067041


In [23]:
cir_final['STD_YMD'] = pd.to_datetime(cir_final['STD_YMD'],format='%Y-%m-%d')

##  Merge {유통 + 소상공인(GS)} data

GS상점수 Mapping method: 
- 2018/03 : 2018/03,04,05
- 2018/06 : 2018/06,07,08
- 2018/09 : 2018/09,10,11
- 2018/12 : 2018/12, 2019/01,02
- 2019/03 : 2019/03,04

In [24]:
# sy = list()
# for i in np.unique(tp_cir['ADMD_NM']):
#     pe1 = tp_cir[tp_cir['ADMD_NM'] == i]
#     cr1 = gs_num3[gs_num3['행정동명'] == i]
    
#     if (pe1['date']<'2018-06-01'):
#         an1 = [cri.iloc[:,-1][0]] * pe1[pe1['date']<'2018-06-01'].shape[1]
#         sy.extend(an1)
#     elif ((pe1['date']>='2018-06-01')&(pe1['date']<'2018-09-01')):
#         an2 = [cri.iloc[:,-1][1]] * pe1[(pe1['date']>='2018-06-01')&(pe1['date']<'2018-09-01')].shape[1]
#         sy.extend(an2)
#     elif ((pe1['date']>='2018-09-01')&(pe1['date']<'2018-12-01')):
#         an3 = [cri.iloc[:,-1][2]] * pe1[(pe1['date']>='2018-09-01')&(pe1['date']<'2018-12-01')].shape[1]
#         sy.extend(an3)
#     elif ((pe1['date']>='2018-12-01')&(pe1['date']<'2019-03-01')):
#         an4 = [cri.iloc[:,-1][3]] * pe1[(pe1['date']>='2018-09-01')&(pe1['date']<'2018-12-01')].shape[1]
#         sy.extend(an4)
#     else:
#         an5 = [cri.iloc[:,-1][4]] * pe1[(pe1['date']>='2018-03-01')].shape[1]
#         sy.extend(an5)


a = '2018-03-31'
b = '2018-06-30'
c = '2018-09-30'
d = '2018-12-31'
e = '2019-03-31'

cir_final['time']=1
cir_final.loc[cir_final['STD_YMD'] < '2018-06-01','time'] = a 
cir_final.loc[((cir_final['STD_YMD']>='2018-06-01')&(cir_final['STD_YMD']<'2018-09-01')),'time']=b
cir_final.loc[((cir_final['STD_YMD']>='2018-09-01')&(cir_final['STD_YMD']<'2018-12-01')),'time'] =c
cir_final.loc[((cir_final['STD_YMD']>='2018-12-01')&(cir_final['STD_YMD']<'2019-03-01')),'time'] =d
cir_final.loc[cir_final['STD_YMD'] >= '2019-03-01','time'] = e   

In [25]:
cir_final.head()

,STD_YMD,ADMD_NM,values,category,category_values,values_ratio,time
0,2018-04-01,가회동,0.741018,10_cate,0.238186,0.176500,2018-03-31
1,2018-04-01,가회동,0.741018,20_cate,0.210444,0.155943,2018-03-31
2,2018-04-01,가회동,0.741018,30_cate,0.412769,0.305870,2018-03-31
3,2018-04-01,가회동,0.741018,40_cate,0.016912,0.012532,2018-03-31
4,2018-04-01,가회동,0.741018,50_cate,0.090471,0.067041,2018-03-31


In [26]:
gs_retail.head()

,상호명,지점명,상권업종대분류명,상권업종중분류명,상권업종소분류명,표준산업분류명,시도명,시군구명,행정동명,법정동명,지번주소,건물명,도로명주소,경도,위도,time
0,GS25광운문화관,NaN,소매,종합소매점,편의점,체인화 편의점,서울특별시,노원구,월계1동,월계동,서울특별시 노원구 월계동 466,광운대학교,서울특별시 노원구 광운로 21,127.057576,37.619816,2018-03-31
1,GS25,명륜스타점,소매,종합소매점,편의점,체인화 편의점,서울특별시,종로구,혜화동,명륜4가,서울특별시 종로구 명륜4가 113-1,대학로스타시스빌딩,서울특별시 종로구 대학로11길 23,127.000401,37.582052,2018-03-31
2,GS25석계그랑빌,NaN,소매,종합소매점,편의점,체인화 편의점,서울특별시,노원구,월계3동,월계동,서울특별시 노원구 월계동 18,그랑빌아파트,서울특별시 노원구 마들로 31,127.067060,37.618720,2018-03-31
3,GS노원중앙점,노원중앙점,소매,종합소매점,편의점,체인화 편의점,서울특별시,노원구,상계6.7동,상계동,서울특별시 노원구 상계동 725-4,영보빌딩,서울특별시 노원구 동일로218길 41,127.063160,37.654172,2018-03-31
4,지에스25상명사랑,NaN,소매,종합소매점,편의점,체인화 편의점,서울특별시,종로구,부암동,홍지동,서울특별시 종로구 홍지동 94-2,NaN,서울특별시 종로구 세검정로 230,126.958506,37.600143,2018-03-31


In [27]:
gs_retail2 = gs_retail.groupby(['행정동명','time']).size().reset_index(level=[0,1])

지수 개발을 위해 '2019-03-31'이전 시점의 관측치가 존재하지 않는 가회동의 GS 계열사 수를 1로 산정한다.

In [28]:
gs_retail2.loc[-1] = ['가회동','2018-12-31',1]
gs_retail2.loc[-2] = ['가회동','2018-09-30',1]
gs_retail2.loc[-3] = ['가회동','2018-06-30',1]
gs_retail2.loc[-4] = ['가회동','2018-03-31',1]
gs_retail2.index = gs_retail2.index +4
gs_retail2 = gs_retail2.sort_index()
gs_retail2 = gs_retail2.rename(columns = {'행정동명': 'ADMD_NM',0:'gs_num'})
gs_retail2

,ADMD_NM,time,gs_num
0,가회동,2018-03-31,1
1,가회동,2018-06-30,1
2,가회동,2018-09-30,1
3,가회동,2018-12-31,1
4,가회동,2019-03-31,1
5,공릉1동,2018-03-31,8
6,공릉1동,2018-06-30,8
7,공릉1동,2018-09-30,8
8,공릉1동,2018-12-31,8
9,공릉1동,2019-03-31,10


In [29]:
len(gs_retail2)

180

In [30]:
gs_distance.rename(columns = {'cri':'time'},inplace=True)

In [31]:
gs = pd.merge(gs_retail2,gs_distance,on=['ADMD_NM','time'])

In [32]:
cir_final_revised = pd.merge(cir_final,gs,on=['ADMD_NM','time'])
cir_final_revised.head()

,STD_YMD,ADMD_NM,values,category,category_values,values_ratio,time,gs_num,min_gs,max_gs,median_gs
0,2018-04-01,가회동,0.741018,10_cate,0.238186,0.176500,2018-03-31,1,0.0,0.0,0.0
1,2018-04-01,가회동,0.741018,20_cate,0.210444,0.155943,2018-03-31,1,0.0,0.0,0.0
2,2018-04-01,가회동,0.741018,30_cate,0.412769,0.305870,2018-03-31,1,0.0,0.0,0.0
3,2018-04-01,가회동,0.741018,40_cate,0.016912,0.012532,2018-03-31,1,0.0,0.0,0.0
4,2018-04-01,가회동,0.741018,50_cate,0.090471,0.067041,2018-03-31,1,0.0,0.0,0.0


## Merge {유동 + 유통 + GS} data

In [33]:
new_tp2.head()

,STD_YMD,ADMD_NM,SEX_CD,AGE_CD,trans_pop
0,20180401,가회동,F,20,1160.64
1,20180401,가회동,F,25,832.24
2,20180401,가회동,F,30,729.24
3,20180401,가회동,F,35,812.86
4,20180401,가회동,F,40,741.41


In [34]:
new_tp2['STD_YMD'] = pd.to_datetime(new_tp2['STD_YMD'],format='%Y%m%d')
cir_final_revised['STD_YMD'] = pd.to_datetime(cir_final_revised['STD_YMD'],format='%Y-%m-%d')

In [35]:
new_tp3 = pd.merge(new_tp2,cir_final_revised,on=['STD_YMD','ADMD_NM'])
len(np.unique(new_tp3['ADMD_NM']))

36

In [36]:
new_tp3.to_csv('가공데이터/merge_유동_유통_edit1.csv',encoding='utf-8')

## Merge {유동 + 유통 + 환경} data

In [37]:
air = pd.read_csv('가공데이터/환경_edit1.csv')

In [38]:
air.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10220 entries, 0 to 10219
Data columns (total 29 columns):
date                    10220 non-null object
year                    10220 non-null int64
quarter                 10220 non-null int64
month                   10220 non-null int64
weekday                 10220 non-null object
day                     10220 non-null int64
district                10220 non-null object
gu                      10220 non-null object
p10_most_freq_day       10220 non-null object
p10_mean_day            10220 non-null float64
p10_25per               10220 non-null float64
p10_median_day          10220 non-null float64
p10_75per               10220 non-null float64
p25_most_freq_day       10220 non-null object
p25_mean_day            10220 non-null float64
p25_25per               10220 non-null float64
p25_median_day          10220 non-null float64
p25_75per               10220 non-null float64
humi_mean_day           10220 non-null float64
humi_median_

In [39]:
air['date'] = pd.to_datetime(air['date'])

In [40]:
new_tp3.rename(columns = {'STD_YMD':'date','ADMD_NM':'district'},inplace=True)

In [41]:
mvp = pd.merge(air,new_tp3,on=['date','district'])

{환경 + 유동 + 유통} data가 26개 동만 남음이 확인 되었다.

In [42]:
len(np.unique(mvp['district']))

26

In [43]:
mvp.to_csv('가공데이터/merge_유통_유동_환경_소상공인_edit2.csv',encoding='utf-8')